In [ ]:
from numpy import NaN
import pandas as pd

# Import the data as a pandas dataframe
progress = pd.read_csv("/content/Vaccine_Progress_Dashboard_Data.csv")

progress.info()
progress['administered_date']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36580 entries, 0 to 36579
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   county                          36580 non-null  object
 1   administered_date               36580 non-null  object
 2   total_doses                     36580 non-null  int64 
 3   cumulative_total_doses          36580 non-null  int64 
 4   pfizer_doses                    36580 non-null  int64 
 5   cumulative_pfizer_doses         36580 non-null  int64 
 6   moderna_doses                   36580 non-null  int64 
 7   cumulative_moderna_doses        36580 non-null  int64 
 8   jj_doses                        36580 non-null  int64 
 9   cumulative_jj_doses             36580 non-null  int64 
 10  partially_vaccinated            36580 non-null  int64 
 11  total_partially_vaccinated      36580 non-null  int64 
 12  fully_vaccinated                36580 non-null

0        2020-01-05
1        2020-03-03
2        2020-07-27
3        2020-07-29
4        2020-07-30
            ...    
36575    2022-03-02
36576    2022-03-03
36577    2022-03-04
36578    2022-03-05
36579    2022-03-06
Name: administered_date, Length: 36580, dtype: object

In [ ]:
# need to fill with california flag for the null values
progress[progress['california_flag'].isnull()]

,county,administered_date,total_doses,cumulative_total_doses,pfizer_doses,cumulative_pfizer_doses,moderna_doses,cumulative_moderna_doses,jj_doses,cumulative_jj_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,booster_recip_count,cumulative_booster_recip_count
0,Alameda,2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
1,Alameda,2020-03-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
2,Alameda,2020-07-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
3,Alameda,2020-07-29,0,1,0,0,0,1,0,0,0,0,0,0,0,1,NaN,0,0
4,Alameda,2020-07-30,0,1,0,0,0,1,0,0,0,0,0,0,0,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8866,Yuba,2020-12-11,0,2,0,0,0,2,0,0,0,0,0,1,0,1,NaN,0,0
8867,Yuba,2020-12-12,0,2,0,0,0,2,0,0,0,0,0,1,0,1,NaN,0,0
8868,Yuba,2020-12-13,0,2,0,0,0,2,0,0,0,0,0,1,0,1,NaN,0,0
8869,Yuba,2020-12-14,0,2,0,0,0,2,0,0,0,0,0,1,0,1,NaN,0,0


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np



# function for building the decision tree
# Return: trained decision tree
def decision_tree_build(train_data, label_data, criterion, min_samples_split, max_depth):
  print("BUILDING DECISION TREE ... ")
  print("Training Data Shape: " + str(train_data.shape))
  dt_clf = DecisionTreeClassifier(criterion = criterion,
                                  min_samples_split = min_samples_split,
                                  max_depth = max_depth)
  dt_clf.fit(train_data, label_data)
  return dt_clf

# function for building the random forest
# Return: trained random forest classifier
def random_forest_build(train_data, label_data, criterion, n_estimators):
  print("BUILDING RANDOM FOREST ... ")
  print("Training Data Shape: " + str(train_data.shape))
  rf_clf = RandomForestClassifier(n_estimators = n_estimators,
                                  criterion = criterion)
  rf_clf.fit(train_data, label_data)
  return rf_clf

# prediction function for the given data and classifier
# Return: output of the prediction
def classifier_prediction(data, trained_classifier):
  print("Feeded Data Shape: " + str(data.shape))
  pred = trained_classifier.predict(data)
  print("Output Prediction Shape: " + str(pred.shape))
  return pred

# show the accuracy of two data set, one should be label and another one is the prediction
# Simply print the result
def output_validation(pred_data, labels):
  # can be replaced by self developed accuracy function
  accuracy = accuracy_score(labels, pred_data)
  print("Acuracy: "+ str(accuracy))
  print("Confusion Matrix: \n" + str(confusion_matrix(labels, pred_data)))



##### testing ######
credit_approval = pd.read_csv("/content/crx.data", header = None)
credit_approval = credit_approval.replace("?", np.NaN)
credit_approval.dropna(axis=0,inplace=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in credit_approval.columns:
  if credit_approval[column].dtype == "object":
    credit_approval[column] = le.fit_transform(credit_approval[column])
credit_approval = credit_approval.drop([11, 13], axis=1)
ca_arr = credit_approval.values

# Segregate features and labels into separate variables
X,y = ca_arr[:,0:12] , ca_arr[:,13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)

scaler = MinMaxScaler()
# transform data
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Example of using decision tree model
decision_model = decision_tree_build(X_train, y_train, criterion="entropy",min_samples_split=4,max_depth=3)
dt_prediction = classifier_prediction(X_test, decision_model)
accuracy = output_validation(dt_prediction, y_test)
print(accuracy)

# Example of using randome forest model
forest_model = random_forest_build(X_train, y_train, n_estimators=20, criterion="entropy")
rf_prediction = classifier_prediction(X_test, forest_model)
accuracy = output_validation(rf_prediction, y_test)
print(accuracy)

BUILDING DECISION TREE ... 
Training Data Shape: (437, 12)
Feeded Data Shape: (216, 12)
Output Prediction Shape: (216,)
Acuracy: 0.875
Confusion Matrix: 
[[96  8]
 [19 93]]
None
BUILDING RANDOM FOREST ... 
Training Data Shape: (437, 12)
Feeded Data Shape: (216, 12)
Output Prediction Shape: (216,)
Acuracy: 0.8935185185185185
Confusion Matrix: 
[[94 10]
 [13 99]]
None
